_한국어로 기계번역됨_


# 그래프의 입력/출력 스키마 정의 방법

<div class="admonition tip">
    <p class="admonition-title">전제 조건</p>
    <p>
        이 가이드는 다음에 대한 기본 지식이 있다고 가정합니다:
        <ul>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/low_level/#multiple-schemas">
                    다중 스키마
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/low_level/#stategraph">
                    상태 그래프
                </a>                
            </li>            
        </ul>
    </p>
</div>

기본적으로 `StateGraph`는 단일 스키마로 작동하며, 모든 노드는 해당 스키마를 사용하여 통신할 것으로 기대됩니다. 그러나 그래프에 대해 별도의 입력 및 출력 스키마를 정의하는 것도 가능합니다.

구별된 스키마가 지정되면 노드 간의 통신을 위해 내부 스키마가 여전히 사용됩니다. 입력 스키마는 제공된 입력이 예상 구조와 일치하는지 확인하고, 출력 스키마는 정의된 출력 스키마에 따라 관련된 정보만 반환하도록 내부 데이터를 필터링합니다.

이 예제에서는 별도의 입력 및 출력 스키마를 정의하는 방법을 살펴보겠습니다.

## 설정

먼저 필요한 패키지를 설치해 보겠습니다.


In [3]:
%%capture --no-stderr
%pip install -U langgraph


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 발견하고 성능을 개선하세요. LangSmith를 사용하면 추적 데이터를 활용하여 LangGraph로 구축한 LLM 앱을 디버그하고 테스트하며 모니터링할 수 있습니다. 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>를 참조하세요.
    </p>
</div>


## 그래프 정의 및 사용


In [6]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


# Define the schema for the input
class InputState(TypedDict):
    question: str


# Define the schema for the output
class OutputState(TypedDict):
    answer: str


# Define the overall schema, combining both input and output
class OverallState(InputState, OutputState):
    pass


# Define the node that processes the input and generates an answer
def answer_node(state: InputState):
    # Example answer and an extra key
    return {"answer": "bye", "question": state["question"]}


# Build the graph with input and output schemas specified
builder = StateGraph(OverallState, input=InputState, output=OutputState)
builder.add_node(answer_node)  # Add the answer node
builder.add_edge(START, "answer_node")  # Define the starting edge
builder.add_edge("answer_node", END)  # Define the ending edge
graph = builder.compile()  # Compile the graph

# Invoke the graph with an input and print the result
print(graph.invoke({"question": "hi"}))


{'answer': 'bye'}


invoke의 출력은 출력 스키마만 포함된다는 점에 유의하세요.
